# imports and defs

In [1]:
import numpy as np
import pandas as pd
import bcsd_module_detrended as bb
import matplotlib.pyplot as plt
import utility_module_2_v_3 as uu
from datetime import datetime


In [2]:
def check_for_nans_infs_zeros(data_sts):
    print('Nans:',np.sum(np.isnan(data_sts)))
    print('Infs:' ,np.sum(np.isinf(data_sts)))
    print('Zeros:',np.sum((data_sts==0)))
    

In [3]:
## check dimensions and plot ts
def data_check(data_sts,lat,lon,Np= 12):
    check_for_nans_infs_zeros(data_sts)
    map_bounds=metadata['data_bounds'] #[50 , 50, 110, 0] #ltrb
    mp_spacing=10

    plt.figure(figsize = (12,2))

    plt.subplot(121)
    uu.map_plot_cyl(np.nanmean(data_sts[:,:,:],axis=0),lat,lon,map_bounds,mp_spacing=10)
    plt.title('Mean')

    plt.subplot(122)
    plt.plot(data_sts[:Np*5,int(lat.shape[0]/2),int(lon.shape[0]/2)])
    plt.plot(data_sts[-Np*5:,int(lat.shape[0]/2),int(lon.shape[0]/2)])


# Exp details

In [4]:
exp_names = ['US_P_BCSD',
'US_P_BCSD_eqm',
'US_T_BCSD',
'US_T_BCSD_eqm',]

SD_types = ['PREC','PREC','TEMP','TEMP']
bc_methods = [bb.gamma_quantile_mapping,bb.empirical_quantile_mapping,bb.normal_quantile_mapping,bb.empirical_quantile_mapping]
var_types = ['pre','pre','tas','tas']

df = pd.DataFrame(list(zip(exp_names, SD_types,bc_methods,var_types)),
               columns =['exp_names', 'SD_types','bc_methods', 'var_types'])
df

,exp_names,SD_types,bc_methods,var_types
0,US_P_BCSD,PREC,<function gamma_quantile_mapping at 0x7fb17b0b...,pre
1,US_P_BCSD_eqm,PREC,<function empirical_quantile_mapping at 0x7fb1...,pre
2,US_T_BCSD,TEMP,<function normal_quantile_mapping at 0x7fb17b0...,tas
3,US_T_BCSD_eqm,TEMP,<function empirical_quantile_mapping at 0x7fb1...,tas


# Runs

In [ ]:


for gcm_name in ['CanESM2','GFDL_CM3','MIROC_ESM','MRI_ESM1','ACCESS1_0','NorESM1_M','CM5A_LR','CNRM_CM5','MPI_ESM_MR']:
    for itr in [0,2]:
        exp_name = df['exp_names'][itr]
        SD_type = df['SD_types'][itr]
        bc_method = df['bc_methods'][itr]
        var_type = df['var_types'][itr]

        path_out = '/home/vikram/BCSD_data/' + exp_name  + '/'
        metadata = np.load(path_out+var_type +'_metadata'+gcm_name + '.npy',allow_pickle=True).item()
        pre_obs = np.load(path_out+var_type +'_obs'+ '.npy')
        pre_gcmh = np.load(path_out+var_type +'_gcm_'+gcm_name+ '.npy')
        pre_gcmf = np.load(path_out+var_type +'_gcm_future_'+gcm_name+ '.npy')

        # rdn=  np.random.randn(pre_gcm.shape[0],pre_gcm.shape[1],pre_gcm.shape[2])
        
        # if SD_type == 'PREC':
        #     rdn =np.exp(rdn) 
        #     print(SD_type)
        # else:
        #     pass
        
        print("Experiment Name:",exp_name)
        print('\n\nData precheck :')
        # data_check(pre_obs, metadata['lat_obs'], metadata['lon_obs'])
        # data_check(pre_gcm, metadata['lat_gcm'], metadata['lon_gcm'])
        # data_check(rdn, metadata['lat_gcm'], metadata['lon_gcm'])

        path_out = metadata['path_out']

        print("\n\nStarted BCSD at:", datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
        print('GCM name : ', gcm_name)
        print('Output path : ',path_out)

        data_bcsd_pred = bb.bcsd(var_type +'_'+gcm_name+'_cru',pre_obs,pre_gcmh,pre_gcmf,metadata,bc_method,temporal_res = "Monthly",sd_type =SD_type,mode = 1)
        # data_bcsd_rnd = bb.bcsd(var_type +'_rnd_0_cru',pre_obs,rdn,rdn,metadata,bc_method,temporal_res = "Monthly",sd_type =SD_type,mode = 1)

        print("Finished at:", datetime.now().strftime("%d/%m/%Y %H:%M:%S"))


        print('\n\nData postcheck :')
        # data_check(pre_obs, metadata['lat_obs'], metadata['lon_obs'])
        # data_check(pre_gcm, metadata['lat_gcm'], metadata['lon_gcm'])
        # data_check(rdn, metadata['lat_gcm'], metadata['lon_gcm'])
        # data_check(data_bcsd_pred, metadata['lat_obs'], metadata['lon_obs'])
        # data_check(data_bcsd_rnd, metadata['lat_obs'], metadata['lon_obs'])


# Exp details

In [ ]:
exp_names = ['US_P_BCSD',
'US_P_BCSD_eqm',
'US_T_BCSD',
'US_T_BCSD_eqm',]

SD_types = ['PREC','PREC','TEMP','TEMP']
bc_methods = [bb.gamma_quantile_mapping,bb.empirical_quantile_mapping,bb.normal_quantile_mapping,bb.empirical_quantile_mapping]
var_types = ['pre','pre','tas','tas']

df = pd.DataFrame(list(zip(exp_names, SD_types,bc_methods,var_types)),
               columns =['exp_names', 'SD_types','bc_methods', 'var_types'])
df['exp_names']

# Runs

In [ ]:


gcm_name ='CanESM2'


for itr in range(len(df)):
    exp_name = df['exp_names'][itr]
    SD_type = df['SD_types'][itr]
    bc_method = df['bc_methods'][itr]
    var_type = df['var_types'][itr]
    
    path_out = '/home/vikram/BCSD_data/' + exp_name  + '/'
    metadata = np.load(path_out+var_type +'_metadata'+gcm_name + '.npy',allow_pickle=True).item()
    pre_obs = np.load(path_out+var_type +'_obs'+ '.npy')
    pre_gcm = np.load(path_out+var_type +'_gcm_'+gcm_name+ '.npy')

    rdn=  np.random.randn(pre_gcm.shape[0],pre_gcm.shape[1],pre_gcm.shape[2])*np.nanmean(pre_obs)
    rdn[rdn<0]=0
    print("Experiment Name:",exp_name)
    print('\n\nData precheck :')
    data_check(pre_obs, metadata['lat_obs'], metadata['lon_obs'])
    data_check(pre_gcm, metadata['lat_gcm'], metadata['lon_gcm'])
    data_check(rdn, metadata['lat_gcm'], metadata['lon_gcm'])

    path_out = metadata['path_out']

    print("\n\nStarted BCSD at:", datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
    print('GCM name : ', gcm_name)
    print('Output path : ',path_out)

    data_bcsd_pred = bb.bcsd(var_type +'_'+gcm_name+'_cru',pre_obs,pre_gcm,pre_gcm,metadata,bc_method,temporal_res = "Monthly",sd_type =SD_type,mode = 1)
    data_bcsd_rnd = bb.bcsd(var_type +'_rnd_new_10_cru',pre_obs,rdn,rdn,metadata,bc_method,temporal_res = "Monthly",sd_type =SD_type,mode = 1)

    print("Finished at:", datetime.now().strftime("%d/%m/%Y %H:%M:%S"))


    print('\n\nData postcheck :')
    # data_check(pre_obs, metadata['lat_obs'], metadata['lon_obs'])
    # data_check(pre_gcm, metadata['lat_gcm'], metadata['lon_gcm'])
    # data_check(rdn, metadata['lat_gcm'], metadata['lon_gcm'])
    data_check(data_bcsd_pred, metadata['lat_obs'], metadata['lon_obs'])
    data_check(data_bcsd_rnd, metadata['lat_obs'], metadata['lon_obs'])


In [ ]:
5